In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import torch
import torch.nn.functional as F

In [3]:
image = cv2.imread("samples/C001H001S0001003001.bmp")

In [4]:
image_t = image.transpose(2, 0, 1)
img3 = torch.from_numpy(image_t.astype(np.float32))
img3 = torch.broadcast_to(img3, (600, 3, 1024, 1024))

In [5]:
tracer_imgs = np.zeros((600, 20, 20, 3), dtype=np.float32)

In [6]:
for t in tqdm(range(600)):
    tracer_imgs[t] = cv2.imread(f"tracer_imgs_light/tracer_{t}.bmp")

100%|██████████| 600/600 [00:00<00:00, 14689.62it/s]


In [7]:
tracer_imgs = tracer_imgs.transpose(0, 3, 1, 2)
tracer_imgs = torch.from_numpy(tracer_imgs)

In [8]:
@torch.no_grad()
def match_template_torch(imgs, templates):
    # res_list = torch.zeros((1, 2))
    res = match_template_torch_core(imgs, templates)

    return res

@torch.jit.script
def match_template_torch_core(img_tensor, template_tensor):
    result1 = F.conv2d(img_tensor, template_tensor, bias=None, stride=1, padding=0)
    result2 = torch.sqrt(torch.sum(template_tensor**2)* torch.nn.functional.conv2d(img_tensor**2, torch.ones_like(template_tensor), bias=None, stride=1, padding=0))

    return result1/result2

In [ ]:
result = match_template_torch(imgs=img3, templates=tracer_imgs)